In [303]:
import pandas as pd
import numpy as np
import json
import re

In [304]:
data=[]
with open('data.json','r') as file:
    data=json.load(file)

Перевести ваш спарсенный json в формат csv таблицу через pandas

In [305]:
data_list = []
names = list(data["data"][0].keys())
for c in data["data"]:
  data_list.append([c[names[0]],c[names[1]],c[names[2]],c[names[3]]])
df = pd.DataFrame(data_list,columns=names)
df.to_csv('hh_data.csv')
df.head()

,title,work experience,salary,region
0,QA Team Lead в сервис транспорта,3–6 лет,з/п не указана,Екатеринбург
1,QA Team Lead в сервис транспорта,3–6 лет,з/п не указана,Новосибирск
2,TechLead / Senior Python Developer,3–6 лет,з/п не указана,"Москва, Деловой центр, Деловой центр, Междунар..."
3,Инженер по исследованиям/Photonics R&D Scientist,1–3 года,з/п не указана,Минск
4,Middle / Senior QA инженер,1–3 года,от 200 000 руб. до вычета налогов,"Москва, Алексеевская, улица Годовикова, 9с10"


Сколько должностей в наборе данных?

In [306]:
regions = df['region']
titles = df['title']
expr = df['work experience']
salary = df['salary']

reg_dict = {}
tit_dict = {}
sal_dict = {}
exp_dict = {}
i=0
for i in range(len(reg)):
  regions[i]=regions[i].split(",")[0]
  if regions[i] in reg_dict.keys(): reg_dict[regions[i]]+=1
  else: reg_dict[regions[i]]=1
  
  if titles[i] in tit_dict.keys(): tit_dict[titles[i]]+=1
  else: tit_dict[titles[i]]=1

  if salary[i] == 'з/п не указана': 
    salary[i] = np.nan
  else: 
    try:
      t = salary[i].replace('\xa0','').lower().split(' ')
    except:
      print(i)
      break
    if 'руб.' in t : coef=1.
    elif 'usd' in t : coef=58.
    elif 'eur' in t : coef=56.
    elif 'kzt' in t : coef=0.12
    else: coef=1000.
    if 'руки' in t : coef=coef*1./0.87
    #elif 'вычета' in t : coef=coef*1.
    salary[i] = round(coef*np.mean(
        [int(c) for c in t if len(re.sub('[^0-9]','',c))>0]),-3)  
  if salary[i] in sal_dict.keys(): sal_dict[salary[i]]+=1
  else: sal_dict[salary[i]]=1

  if expr[i]=='не требуется':expr[i]=0.
  elif expr[i]=='1–3 года':expr[i]=2.0
  elif expr[i]=='3–6 лет':expr[i]=4.5
  elif expr[i]=='более 6 лет':expr[i]=7
  if expr[i] in exp_dict.keys(): exp_dict[expr[i]]+=1
  else: exp_dict[expr[i]]=1

print(f'Всего записей {len(regions)}, в них разных групп \n '+
      f'Регинов {len(reg_dict)}, Опыта {len(exp_dict)}, Зарплат {len(sal_dict)}, '+
      f'Названий вакансий {len(tit_dict)}.')

Всего записей 1984, в них разных групп 
 Регинов 150, Опыта 4, Зарплат 157, Названий вакансий 1307.


In [307]:
df.to_csv('hh_data_2.csv')
df

,title,work experience,salary,region
0,QA Team Lead в сервис транспорта,4.5,NaN,Екатеринбург
1,QA Team Lead в сервис транспорта,4.5,NaN,Новосибирск
2,TechLead / Senior Python Developer,4.5,NaN,Москва
3,Инженер по исследованиям/Photonics R&D Scientist,2.0,NaN,Минск
4,Middle / Senior QA инженер,2.0,200000.0,Москва
...,...,...,...,...
1979,Разработчик C (Linux kernel space),4.5,NaN,Санкт-Петербург
1980,Администратор БД,4.5,NaN,Москва
1981,Программист 1С,4.5,NaN,Минск
1982,Коммерческий аналитик (senior),2.0,NaN,Москва


Сгруппируйте вакансии по направлениями (DS, DE, Software Engenering, etc.)

In [308]:
def WordIn(st,pt):
  return str(st).replace('/',' ').lower().find(pt) > -1
def GroupW(st):
  if max(WordIn(st,'данны'),
       WordIn(st,'data')) and max(WordIn(st,'analyt'),
                                         WordIn(st,'аналит'),
                                         WordIn(st,'scien')) : t = 'DS'
  elif max(WordIn(st,'данны'),
         WordIn(st,'data'),
         WordIn(st,'elk'),
         WordIn(st,'devops')) and max(WordIn(st,'enginee'),
                                           WordIn(st,'инжен')) : t = 'DE'
  elif max(WordIn(st,'разработчик'),
           WordIn(st,'программист'),
         WordIn(st,'developer')) or max(WordIn(st,'с++'),
                                           WordIn(st,'pyth'),
                                           WordIn(st,'djang')) : t = 'Developer'
  elif max(WordIn(st,'тестир'),
         WordIn(st,'qa')): t = 'QA'
  elif max(WordIn(st,'продукт'),
       WordIn(st,'product')) and max(WordIn(st,'analyt'),
                                         WordIn(st,'аналит')) : t = 'Продуктовый Аналитик'
  else:
    t='Others'
  return t


In [309]:
df['group']=""
for i in range(len(df['title'])):
  t=GroupW(df['title'][i])
  df['group'][i]=t
df

,title,work experience,salary,region,group
0,QA Team Lead в сервис транспорта,4.5,NaN,Екатеринбург,QA
1,QA Team Lead в сервис транспорта,4.5,NaN,Новосибирск,QA
2,TechLead / Senior Python Developer,4.5,NaN,Москва,Developer
3,Инженер по исследованиям/Photonics R&D Scientist,2.0,NaN,Минск,Others
4,Middle / Senior QA инженер,2.0,200000.0,Москва,QA
...,...,...,...,...,...
1979,Разработчик C (Linux kernel space),4.5,NaN,Санкт-Петербург,Developer
1980,Администратор БД,4.5,NaN,Москва,Others
1981,Программист 1С,4.5,NaN,Минск,Developer
1982,Коммерческий аналитик (senior),2.0,NaN,Москва,Others


Какая средняя и медианная зарплата по группам вакансий?

Какая самая высокооплачиваемая из групп вакансий, исходя из их средних зарплат?

In [310]:
df.groupby('group').salary.agg([len,
                                np.mean,
                                np.median]).sort_values('median',ascending=False)

,len,mean,median
group,,,
DE,148,262781.250000,270000.0
Продуктовый Аналитик,8,178000.000000,178000.0
Developer,1060,573695.214106,172000.0
QA,109,174590.909091,152500.0
DS,36,198857.142857,123500.0
Others,623,125658.181818,37000.0


In [311]:
df.groupby('group')['work experience'].agg([len,
                                            np.mean,
                                            np.median]).sort_values('len',ascending=False)

,len,mean,median
group,,,
Developer,1060,3.239151,4.5
Others,623,2.757624,2.0
DE,148,3.658784,4.5
QA,109,3.096330,2.0
DS,36,3.555556,4.5
Продуктовый Аналитик,8,2.375000,2.0


Какая средняя и медианная зарплата по каждому региону?

In [312]:
df.groupby('region')['salary'].agg([len,
                                    np.mean,
                                    np.median]).sort_values('len',ascending=False)

/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/python3.7/dist-packages/numpy/lib/nanfunctions.py:1117: RuntimeWarning: Mean of empty slice
  return np.nanmean(a, axis, out=out, keepdims=keepdims)
/usr/local/lib/p

,len,mean,median
region,,,
Москва,916,2.172239e+05,200000.0
Санкт-Петербург,268,1.932561e+05,172000.0
Минск,47,2.194500e+06,2241500.0
Екатеринбург,43,1.745294e+05,172000.0
Новосибирск,43,1.474211e+05,138000.0
...,...,...,...
Новомосковск,1,6.000000e+04,60000.0
Муром,1,NaN,NaN
Люберцы,1,1.150000e+05,115000.0


Какое процентное соотношение каждого региона по вакансиям от всех вакансий?

In [313]:
df.groupby('region').title.count().sort_values(ascending=False)/len(df.title)

region
Москва             0.461694
Санкт-Петербург    0.135081
Минск              0.023690
Екатеринбург       0.021673
Новосибирск        0.021673
                     ...   
Новомосковск       0.000504
Муром              0.000504
Люберцы            0.000504
Латвия             0.000504
посёлок Шушары     0.000504
Name: title, Length: 150, dtype: float64

Какая корреляция уровня опыта от зарплаты?

In [314]:
df['work experience']=pd.to_numeric(df['work experience'])
df['salary']=pd.to_numeric(df['salary'])
t0 = pd.isna(df['salary'])==False

for gr in set(df.group.values):
  t = (df['group']==gr) * t0 == 1
  #print(gr,sum(t))
  if(sum(t)>1):
    ex=df['work experience'][t]
    sl=df['salary'][t]
    print(f' Корреляция в группе {gr} = {np.corrcoef(ex,sl)[0,1]}')
  else:
    print(f' В группе {gr} указана зарплата для {sum(t)} вакансии')


 В группе Продуктовый Аналитик указана зарплата для 1 вакансии
 Корреляция в группе Others = 0.4667802675789619
 Корреляция в группе DE = 0.33792278146679594
 Корреляция в группе DS = 0.49105186145666757
 Корреляция в группе Developer = 0.04792785696262304
 Корреляция в группе QA = 0.37564000282598764


Какие 10 наиболее часто встречающихся должностей?

In [315]:
df.groupby('title').title.count().sort_values(ascending=False)[:10]

title
Специалист службы поддержки с техническими знаниями в Яндекс Поиск    69
Специалист службы поддержки (с техническими знаниями)                 31
Специалист службы поддержки с техническими знаниями                   29
Инженер-программист                                                   26
Ведущий сетевой инженер в ЦОД                                         20
Python Developer                                                      15
DevOps Engineer                                                       14
Программист Python                                                    13
Программист                                                           13
DevOps инженер                                                        13
Name: title, dtype: int64